# Dependencies

In [1]:
import pandas as pd

# Data Import and Cleanup

In [2]:
movies_meta = pd.read_csv("datasets/the-movies-dataset/movies_metadata.csv")

# filter out foreign movies (not en or de)
movies_meta = movies_meta[movies_meta.original_language.isin(["en", "de"])]

# filter movies with NA in critical columns
for column in ["imdb_id", "title", "release_date", "popularity"]:
    movies_meta = movies_meta[movies_meta[column].isna() == False]

# filter adult movies
movies_meta = movies_meta[movies_meta.adult == "False"]

# sort the list by popularity
movies_meta["popularity"] = pd.to_numeric(movies_meta.popularity)
movies_meta = movies_meta.sort_values("popularity", ascending=False)

# drop unnecessary columns
movies_meta = movies_meta.drop(["adult", "belongs_to_collection", "id"], axis=1)

movies_meta.head(2)

,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
30700,74000000,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",http://www.minionsmovie.com/,tt2293640,en,Minions,"Minions Stuart, Kevin and Bob are recruited by...",547.488298,/q0R4crx2SehcEEQEkYObktdeFy.jpg,"[{'name': 'Universal Pictures', 'id': 33}, {'n...",...,2015-06-17,1.156731e+09,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"Before Gru, they had a history of bad bosses",Minions,False,6.4,4729.0
33356,149000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.warnerbros.com/wonder-woman,tt0451279,en,Wonder Woman,An Amazon princess comes to the world of Man t...,294.337037,/imekS7f1OuHyUP2LAiTEM0zBzUz.jpg,"[{'name': 'Dune Entertainment', 'id': 444}, {'...",...,2017-05-30,8.205804e+08,141.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Released,Power. Grace. Wisdom. Wonder.,Wonder Woman,False,7.2,5025.0


In [3]:
# find and purge duplicated imdb ids
movies_meta["is_duplicate"] = movies_meta.duplicated(["imdb_id"])
movies_meta = movies_meta[~movies_meta.is_duplicate]
max(movies_meta.imdb_id.value_counts())

1

In [4]:
# TODO: we might actually want to use these columns, but we do not save them to the DB in a first iteration!
movies_meta = movies_meta.drop(["genres", "production_companies", "spoken_languages", "production_countries", "is_duplicate"], axis=1)

# Segmentation based on Release Date

In [5]:
def assign_date_bucket(row):
    as_dt = pd.to_datetime(row["release_date"])
    year = as_dt.year
    
    # before 1990
    if year < 1990:
        return 0
    
    # 1990-2005
    elif year < 2005:
        return 1
    
    # 2005-2020
    return 2

movies_meta["date_segment"] = movies_meta.apply(assign_date_bucket, axis=1)
movies_meta.head(2)

,budget,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,date_segment
30700,74000000,http://www.minionsmovie.com/,tt2293640,en,Minions,"Minions Stuart, Kevin and Bob are recruited by...",547.488298,/q0R4crx2SehcEEQEkYObktdeFy.jpg,2015-06-17,1.156731e+09,91.0,Released,"Before Gru, they had a history of bad bosses",Minions,False,6.4,4729.0,2
33356,149000000,http://www.warnerbros.com/wonder-woman,tt0451279,en,Wonder Woman,An Amazon princess comes to the world of Man t...,294.337037,/imekS7f1OuHyUP2LAiTEM0zBzUz.jpg,2017-05-30,8.205804e+08,141.0,Released,Power. Grace. Wisdom. Wonder.,Wonder Woman,False,7.2,5025.0,2


In [6]:
# movies_meta.info()

In [7]:
movies_until_1990 = movies_meta[movies_meta["date_segment"] == 0]
movies_until_1990.head(2)

,budget,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,date_segment
536,28000000,http://www.warnerbros.com/blade-runner,tt0083658,en,Blade Runner,In the smog-choked dystopian Los Angeles of 20...,96.272374,/p64TtbZGCElxQHpAMWmDHkWJlH2.jpg,1982-06-25,33139618.0,117.0,Released,Man has made his match... now it's his problem.,Blade Runner,False,7.9,3833.0,0
256,11000000,http://www.starwars.com/films/star-wars-episod...,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,42.149697,/btTdmkgIvOi0FFip1sPuZI2oQG6.jpg,1977-05-25,775398007.0,121.0,Released,"A long time ago in a galaxy far, far away...",Star Wars,False,8.1,6778.0,0


In [8]:
movies_until_2005 = movies_meta[movies_meta["date_segment"] == 1]
movies_until_2005.head(2)

,budget,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,date_segment
292,8000000,NaN,tt0110912,en,Pulp Fiction,"A burger-loving hit man, his philosophical par...",140.950236,/dM2w364MScsjFf8pfMbaWUcWrR.jpg,1994-09-10,213928762.0,154.0,Released,Just because you are a character doesn't mean ...,Pulp Fiction,False,8.3,8670.0,1
2843,63000000,http://www.foxmovies.com/movies/fight-club,tt0137523,en,Fight Club,A ticking-time-bomb insomniac and a slippery s...,63.869599,/adw6Lq9FiC9zjYEpOqfq03ituwp.jpg,1999-10-15,100853753.0,139.0,Released,Mischief. Mayhem. Soap.,Fight Club,False,8.3,9678.0,1


In [9]:
movies_until_2020 = movies_meta[movies_meta["date_segment"] == 2]
movies_until_2020.head(2)

,budget,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,date_segment
30700,74000000,http://www.minionsmovie.com/,tt2293640,en,Minions,"Minions Stuart, Kevin and Bob are recruited by...",547.488298,/q0R4crx2SehcEEQEkYObktdeFy.jpg,2015-06-17,1.156731e+09,91.0,Released,"Before Gru, they had a history of bad bosses",Minions,False,6.4,4729.0,2
33356,149000000,http://www.warnerbros.com/wonder-woman,tt0451279,en,Wonder Woman,An Amazon princess comes to the world of Man t...,294.337037,/imekS7f1OuHyUP2LAiTEM0zBzUz.jpg,2017-05-30,8.205804e+08,141.0,Released,Power. Grace. Wisdom. Wonder.,Wonder Woman,False,7.2,5025.0,2


# Extract Top50

In [10]:
top50_until_1990 = movies_until_1990.iloc[0:50]
top50_until_1990.head(2)

,budget,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,date_segment
536,28000000,http://www.warnerbros.com/blade-runner,tt0083658,en,Blade Runner,In the smog-choked dystopian Los Angeles of 20...,96.272374,/p64TtbZGCElxQHpAMWmDHkWJlH2.jpg,1982-06-25,33139618.0,117.0,Released,Man has made his match... now it's his problem.,Blade Runner,False,7.9,3833.0,0
256,11000000,http://www.starwars.com/films/star-wars-episod...,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,42.149697,/btTdmkgIvOi0FFip1sPuZI2oQG6.jpg,1977-05-25,775398007.0,121.0,Released,"A long time ago in a galaxy far, far away...",Star Wars,False,8.1,6778.0,0


In [11]:
top50_until_2005 = movies_until_2005.iloc[0:50]
top50_until_2005.head(2)

,budget,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,date_segment
292,8000000,NaN,tt0110912,en,Pulp Fiction,"A burger-loving hit man, his philosophical par...",140.950236,/dM2w364MScsjFf8pfMbaWUcWrR.jpg,1994-09-10,213928762.0,154.0,Released,Just because you are a character doesn't mean ...,Pulp Fiction,False,8.3,8670.0,1
2843,63000000,http://www.foxmovies.com/movies/fight-club,tt0137523,en,Fight Club,A ticking-time-bomb insomniac and a slippery s...,63.869599,/adw6Lq9FiC9zjYEpOqfq03ituwp.jpg,1999-10-15,100853753.0,139.0,Released,Mischief. Mayhem. Soap.,Fight Club,False,8.3,9678.0,1


In [12]:
top50_until_2020 = movies_until_2020.iloc[0:50]
top50_until_2020.head(2)

,budget,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,date_segment
30700,74000000,http://www.minionsmovie.com/,tt2293640,en,Minions,"Minions Stuart, Kevin and Bob are recruited by...",547.488298,/q0R4crx2SehcEEQEkYObktdeFy.jpg,2015-06-17,1.156731e+09,91.0,Released,"Before Gru, they had a history of bad bosses",Minions,False,6.4,4729.0,2
33356,149000000,http://www.warnerbros.com/wonder-woman,tt0451279,en,Wonder Woman,An Amazon princess comes to the world of Man t...,294.337037,/imekS7f1OuHyUP2LAiTEM0zBzUz.jpg,2017-05-30,8.205804e+08,141.0,Released,Power. Grace. Wisdom. Wonder.,Wonder Woman,False,7.2,5025.0,2


# Ensure Consistency with Poster-Service

In [13]:
import requests

In [14]:
top50_all = pd.concat([top50_until_1990, top50_until_2005, top50_until_2020])

In [15]:
top50_all.head(2)

,budget,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,date_segment
536,28000000,http://www.warnerbros.com/blade-runner,tt0083658,en,Blade Runner,In the smog-choked dystopian Los Angeles of 20...,96.272374,/p64TtbZGCElxQHpAMWmDHkWJlH2.jpg,1982-06-25,33139618.0,117.0,Released,Man has made his match... now it's his problem.,Blade Runner,False,7.9,3833.0,0
256,11000000,http://www.starwars.com/films/star-wars-episod...,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,42.149697,/btTdmkgIvOi0FFip1sPuZI2oQG6.jpg,1977-05-25,775398007.0,121.0,Released,"A long time ago in a galaxy far, far away...",Star Wars,False,8.1,6778.0,0


In [24]:
def check_omdb_existence(row):
    try:
        result = requests.get(f"http://localhost:3000/?id={row['imdb_id']}")
        return result.status_code == 200
    except:
        return False

top50_all["omdb_consistent"] = top50_all.apply(check_omdb_existence, axis=1)

In [25]:
top50_all[~top50_all["omdb_consistent"]]

,budget,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,date_segment,omdb_consistent
26566,200000000,http://marvel.com/movies/movie/221/guardians_o...,tt3896198,en,Guardians of the Galaxy Vol. 2,The Guardians must fight to keep their newfoun...,185.330992,/y4MBh0EjBlMuOzv9axM4qJlmhzz.jpg,2017-04-19,863416141.0,137.0,Released,Obviously.,Guardians of the Galaxy Vol. 2,False,7.6,4858.0,2,False


# Hydrate Database

In [26]:
# conda install sqlalchemy, psycopg2
# pip install python-dotenv

import os

from sqlalchemy import create_engine
from dotenv import load_dotenv
load_dotenv()

True

In [27]:
engine = create_engine(os.getenv("DATABASE_URL"), echo=True)

In [30]:
top50_all = top50_all.set_index("imdb_id")
top50_all[~top50_all["omdb_consistent"]].to_sql("movies", con=engine, if_exists="replace")

2020-03-22 20:22:40,898 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-03-22 20:22:40,899 INFO sqlalchemy.engine.base.Engine {'name': 'movies'}
2020-03-22 20:22:41,024 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-03-22 20:22:41,026 INFO sqlalchemy.engine.base.Engine {'name': 'movies'}
2020-03-22 20:22:41,142 INFO sqlalchemy.engine.base.Engine SELECT c.relname FROM pg_class c JOIN pg_namespace n ON n.oid = c.relnamespace WHERE n.nspname = %(schema)s AND c.relkind in ('r', 'p')
2020-03-22 20:22:41,143 INFO sqlalchemy.engine.base.Engine {'schema': 'public'}
2020-03-22 20:22:41,217 INFO sqlalchemy.engine.base.Engine 
            SELECT c.oid
            FROM pg_catalog.pg_class c
            LEFT JOIN pg_catalog.pg_namespace n